# Практическая работа №2

## Импорт необходимых библиотек

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_classif, RFE, SelectKBest, chi2
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from functions import rock_rating_curve_charts
from typing import Dict

In [2]:
dataframe_train = pd.read_csv("../data/train.csv")
dataframe_test = pd.read_csv("../data/test.csv")
dataframe_train.head(10)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,213892,Male,48,1,35.0,1,1-2 Year,No,24922.0,26.0,86,0
1,269011,Male,58,1,28.0,0,1-2 Year,Yes,63541.0,26.0,57,0
2,31464,Male,44,1,28.0,1,1-2 Year,No,30027.0,26.0,286,0
3,86379,Female,22,1,8.0,0,< 1 Year,Yes,35039.0,152.0,180,0
4,456843,Male,24,1,6.0,1,< 1 Year,No,24149.0,152.0,17,0
5,195890,Female,49,1,28.0,0,1-2 Year,Yes,2630.0,157.0,156,0
6,111681,Male,28,1,6.0,1,< 1 Year,No,31154.0,152.0,150,0
7,427068,Male,22,1,12.0,1,< 1 Year,No,27956.0,152.0,174,0
8,337519,Female,25,1,14.0,1,< 1 Year,No,32658.0,152.0,32,0
9,12666,Male,47,1,0.0,0,1-2 Year,Yes,2630.0,26.0,79,0


Выведем информацию о таблицах.

In [3]:
dataframe_train.info()
dataframe_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229292 entries, 0 to 229291
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    229292 non-null  int64  
 1   Gender                229292 non-null  object 
 2   Age                   229292 non-null  int64  
 3   Driving_License       229292 non-null  int64  
 4   Region_Code           229292 non-null  float64
 5   Previously_Insured    229292 non-null  int64  
 6   Vehicle_Age           229292 non-null  object 
 7   Vehicle_Damage        229292 non-null  object 
 8   Annual_Premium        229292 non-null  float64
 9   Policy_Sales_Channel  229292 non-null  float64
 10  Vintage               229292 non-null  int64  
 11  Response              229292 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 21.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152862 entries, 0 to 152861
Data columns (total 

In [4]:
dataframe_train.dtypes

id                        int64
Gender                   object
Age                       int64
Driving_License           int64
Region_Code             float64
Previously_Insured        int64
Vehicle_Age              object
Vehicle_Damage           object
Annual_Premium          float64
Policy_Sales_Channel    float64
Vintage                   int64
Response                  int64
dtype: object

In [5]:
dataframe_train.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

Из вывода видно, что в столбцах нет пустых значений. Разделим наши признаки на категориальные и числовые.

In [6]:
dataframe_categorical = ["Gender", "Vehicle_Age", "Vehicle_Damage"]
dataframe_numerical = ["Age", "Driving_License", "Previously_Insured", "Region_Code", "Annual_Premium", "Policy_Sales_Channel", "Vintage"]
dataframe_all = dataframe_numerical + dataframe_categorical
result_column = "Response"

Так как признак id не играет важной роли в предсказании, то можем удалить его из датафрейма.

In [7]:
dataframe_train.drop("id", axis=1, inplace=True)

In [8]:
for column in dataframe_categorical:
    print(dataframe_train[column].unique())

['Male' 'Female']
['1-2 Year' '< 1 Year' '> 2 Years']
['No' 'Yes']


In [9]:
for column in dataframe_numerical:
    print(dataframe_train[column].describe())

count    229292.000000
mean         38.554987
std          15.231658
min          20.000000
25%          25.000000
50%          36.000000
75%          49.000000
max          85.000000
Name: Age, dtype: float64
count    229292.000000
mean          0.998138
std           0.043114
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: Driving_License, dtype: float64
count    229292.000000
mean          0.489062
std           0.499881
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: Previously_Insured, dtype: float64
count    229292.000000
mean         26.420207
std          13.191474
min           0.000000
25%          15.000000
50%          28.000000
75%          35.000000
max          52.000000
Name: Region_Code, dtype: float64
count    229292.000000
mean      30730.076549
std       17054.686028
min        2630.000000
25%       24564.000000
50%       

Произведём кодирование категориальных признаков

In [10]:
label_encoder = LabelEncoder()
for column in dataframe_categorical:
    dataframe_train[column] = label_encoder.fit_transform(dataframe_train[column])
    dataframe_test[column] = label_encoder.fit_transform(dataframe_test[column])
    dataframe_train[column] = dataframe_train[column].astype("category")
    dataframe_test[column] = dataframe_test[column].astype("category")
    print(dataframe_train[column].unique())

[1, 0]
Categories (2, int64): [0, 1]
[0, 1, 2]
Categories (3, int64): [0, 1, 2]
[0, 1]
Categories (2, int64): [0, 1]


Произведём нормирование всех числовых признаков

In [11]:
min_max_scaler = MinMaxScaler()
dataframe_train[dataframe_numerical] = min_max_scaler.fit_transform(dataframe_train[dataframe_numerical])
dataframe_test[dataframe_numerical] = min_max_scaler.fit_transform(dataframe_test[dataframe_numerical])
dataframe_train.describe()

,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,229292.000000,229292.000000,229292.000000,229292.000000,229292.000000,229292.000000,229292.000000,229292.000000
mean,0.285461,0.998138,0.508081,0.489062,0.052276,0.684714,0.498943,0.164079
std,0.234333,0.043114,0.253682,0.499881,0.031728,0.335236,0.289713,0.370348
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.076923,1.000000,0.288462,0.000000,0.040805,0.154321,0.249135,0.000000
50%,0.246154,1.000000,0.538462,0.000000,0.054097,0.858025,0.498270,0.000000
75%,0.446154,1.000000,0.673077,1.000000,0.068535,0.932099,0.750865,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Отбор информативных признаков
Отбор информативных признаков - это процесс выбора наиболее значимых признаков из набора данных для использования в модели машинного обучения. Он помогает улучшить качество модели и сократить время её обучения. Если в наборе данных есть много признаков, которые не имеют большого влияния на результаты моделирования, то отбрасывание этих признаков улучшит качество модели. Существует несколько методов отбора информативных признаков, таких как обертывание, фильтрация и встроенные методы.

Этот код разделяет данные на обучающий и тестовый наборы с помощью функции train_test_split из библиотеки sklearn.model_selection. Функция train_test_split разделяет данные на обучающий и тестовый наборы в соотношении 70/30.

In [12]:
x = dataframe_train[dataframe_all]
y = dataframe_train[result_column]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=7)
number_best_features = 5

### Метод фильтрации
Метод фильтрации - это метод отбора признаков на основе их статистических свойств. Он основан на оценке значимости каждого признака по отдельности и выборе наиболее значимых признаков. Примерами таких методов можно назвать корреляционный анализ и анализ дисперсии.
В этом коде используется метод SelectKBest из библиотеки scikit-learn для выбора лучших признаков из набора данных. Переменная skb создает объект SelectKBest с параметром k=number_best_features, указывающий количество наилучших признаков, которые будут выбраны. Метод fit() применяется к skb и используется для обучения модели на данных x и y. Метод get_feature_names_out() возвращает список лучших признаков.

In [13]:
skb = SelectKBest(k=number_best_features)
top = skb.fit(x, y)
list_of_best_features = top.get_feature_names_out()
print(list_of_best_features)

['Age' 'Previously_Insured' 'Policy_Sales_Channel' 'Vehicle_Age'
 'Vehicle_Damage']


### Метод обёртки
Метод обертывания - это метод отбора признаков на основе обучения модели машинного обучения с использованием различных наборов признаков. Он основан на оценке качества модели для каждого набора признаков и выборе наилучшего набора.
В коде используется метод обертывания Recursive Feature Elimination из библиотеки scikit-learn для выбора лучших признаков из набора данных. Переменная clf создает объект LinearRegression(), который используется в качестве модели машинного обучения.

Переменная rfe создает объект RFE с параметрами clf, n_features_to_select=n_features_select и step=1. Параметр number_best_features указывает количество лучших признаков, которые будут выбраны. Параметр step указывает шаг уменьшения количества признаков на каждой итерации. Метод fit() применяется к объекту rfe и используется для обучения модели на данных x и y. Метод get_feature_names_out() возвращает список лучших признаков.

In [14]:
lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=number_best_features, step=1)
top_five = rfe.fit(x, y)
selected_features_RFE = top_five.get_feature_names_out()
print(selected_features_RFE)

['Driving_License' 'Previously_Insured' 'Annual_Premium'
 'Policy_Sales_Channel' 'Vehicle_Damage']


### Встроенные методы
Встроенные методы - это методы отбора признаков, которые включены в процесс обучения модели машинного обучения. Они основаны на оценке значимости каждого признака в процессе обучения модели и выборе наиболее значимых признаков.
В этом коде используется метод SelectKBest из библиотеки Scikit-learn для выбора лучших признаков из набора данных. Переменная selector создает объект SelectKBest с параметрами chi2 и k=number_best_features. Параметр chi2 указывает на метод, который будет использоваться для оценки значимости признаков. Параметр k указывает количество лучших признаков, которые будут выбраны. Метод fit_transform() применяется к объекту selector и используется для обучения модели на данных x и y. Метод get_feature_names_out() возвращает список лучших признаков.

In [15]:
selector = SelectKBest(chi2, k=number_best_features)
selector.fit_transform(x, y)
selected_features_chi2 = selector.get_feature_names_out()
print(selected_features_chi2)

['Age' 'Previously_Insured' 'Policy_Sales_Channel' 'Vehicle_Age'
 'Vehicle_Damage']


### Сравнение полученных результатов
Используем метод Gaussian Naive Bayes из библиотеки Scikit-learn для обучения модели машинного обучения на данных x_train и y_train. Он реализует гауссовский наивный байесовский алгоритм для классификации. Переменная model создает объект GaussianNB() и применяет к нему метод fit() с параметрами x_train и y_train, это обучает модель на данных. Переменная masks хранит список масок, каждая из которых содержит True для каждого признака в x_train. Переменная scoring хранит список оценок ROC-AUC для каждой маски. Метод predict_proba() применяется к модели и используется для предсказания вероятности классов на данных x_test. Метод roc_auc_score() вычисляет оценку ROC-AUC для предсказанных вероятностей и фактических значений y_test.

Рассчитаем значение оценки для изначального фрейма данных.

In [16]:
masks = scoring = tuple()
model = GaussianNB().fit(x_train, y_train)
masks += ([True for _ in range(x_train.shape[1])],)
scoring += (roc_auc_score(y_test, model.predict_proba(x_test)[:, 1]),)
print("ROC-AUC: " + str(round(scoring[-1], 7)))

ROC-AUC: 0.8644921


Создаём словарь values_dict, который содержит три метода отбора признаков: chi2, rfe и mutual_inf, проходим по каждому методу отбора признаков и выполняем подсчет оценки. Создаём объект GaussianNB() и применяем к нему метод fit() с параметрами selector.transform(x_train) и y_train. Это обучит модель на данных, отобранных методом selector.

In [17]:
values_dict = {
    "chi2": skb,
    "rfe": rfe,
    "mutual_inf": selector,
}

for selector_name, selector in values_dict.items():
    model = GaussianNB().fit(selector.transform(x_train), y_train)
    masks += (selector.get_support(),)
    scoring = (roc_auc_score(y_test, model.predict_proba(selector.transform(x_test))[:, 1]),)
    print("ROC-AUC: " + str(round(scoring[-1], 7)))

ROC-AUC: 0.8636158
ROC-AUC: 0.8459675
ROC-AUC: 0.8636158


Исходя из результатов можно сделать вывод, что первый и второй метод отбора признаков являются наилучшими.

## Удаление незначимых признаков

In [18]:
dataframe_train = dataframe_train.loc[:, np.append(list_of_best_features, result_column)]  # loc используется для выборки строк и столбцов
dataframe_test = dataframe_test.loc[:, list_of_best_features]

dataframe_all = list_of_best_features

x = dataframe_train[dataframe_all]
y = dataframe_train[result_column]

In [19]:
dataframe_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229292 entries, 0 to 229291
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   Age                   229292 non-null  float64 
 1   Previously_Insured    229292 non-null  float64 
 2   Policy_Sales_Channel  229292 non-null  float64 
 3   Vehicle_Age           229292 non-null  category
 4   Vehicle_Damage        229292 non-null  category
 5   Response              229292 non-null  int64   
dtypes: category(2), float64(3), int64(1)
memory usage: 7.4 MB


## Сэмплирование данных
Семплирование данных - это метод корректировки обучающей выборки с целью балансировки распределения классов в исходном наборе данных.

In [20]:
# sampling = ((x, y), RandomOverSampler().fit_resample(x, y), RandomUnderSampler().fit_resample(x, y))

over_sampler = RandomOverSampler()
under_sampler = RandomUnderSampler()

x_over, y_over = over_sampler.fit_resample(x, y)
x_under, y_under = under_sampler.fit_resample(x, y)

scoring = tuple()

print("ROC-AUC:")

for sx, sy in [(x, y), (x_over, y_over), (x_under, y_under)]:
    scoring += (round(max(cross_val_score(GaussianNB(), sx, sy, scoring="roc_auc", n_jobs=-1)), 3), )
    print(scoring[-1])

ROC-AUC:
0.866
0.865
0.869


Самое лучшее значение получилось у набора, где был under-sampling.

In [22]:
x, y = x_under, y_under
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=7)  # random_state устанавливает начальное значение для генератора случайных чисел.

## Построение моделей классификаторов
Построение моделей классификаторов - это процесс создания алгоритма машинного обучения, который может классифицировать данные на основе определенных признаков. Есть множество методов и алгоритмов для построения моделей классификаторов в зависимости от типа данных и задачи классификации:
1. Логистическая регрессия
2. Метод ближайших соседей
3. Наивный байесовский классификатор
4. Дискриминантный анализ (линейный дискриминантный анализ,
квадратичный дискриминантный анализ)
5. Машина опорных векторов.

In [ ]:
def create_train_model(clf, param_grid=None) -> GridSearchCV:
    if param_grid is None:
        param_grid = {}
    print("Создание и обучение модели...")
    """Создать и обучить модель с подбором параметров."""
    new_model = GridSearchCV(clf, param_grid, n_jobs=-1, scoring="roc_auc").fit(x_train, y_train)
    if param_grid:
        print(new_model.best_estimator_)
    print("ROC-AUC: " + str(round(new_model.score(x_test, y_test), 6)))
    return new_model

### Логистическая регрессия

In [ ]:
param_grid = {
    "penalty": ["l1", "l2"],
    "solver": ["saga", "liblinear"],
    "max_iter": range(1, 10)
}

lg = create_train_model(LogisticRegression(), param_grid)